In [1]:
pip install qiskit 
pip install numpy


SyntaxError: invalid syntax (2732369214.py, line 1)

In [3]:
from qiskit import QuantumCircuit, QuantumRegister
import numpy as np

# Define the number of qubits
n_qubits = 2
qr = QuantumRegister(n_qubits, 'q')
circuit = QuantumCircuit(qr)

# Step 1: Prepare the reference state |10>
# Qubit 0 is occupied (|1>), qubit 1 is virtual (|0>)
circuit.x(0)

# Step 2: Define the variational parameter theta
theta = 0.1

# Step 3: Apply e^{theta/2 * Y1 X0}
# Basis change: X0 -> Z0 with H, Y1 -> Z1 with Rx(pi/2)
circuit.h(0)
circuit.rx(np.pi/2, 1)
# Implement e^{i (theta/2) Z1 Z0} = CNOT_{1,0} Rz(-2 * theta/2) CNOT_{1,0}
circuit.cx(1, 0)
circuit.rz(-theta, 0)  # -2 * (theta/2) = -theta
circuit.cx(1, 0)
# Inverse basis change
circuit.rx(-np.pi/2, 1)
circuit.h(0)

# Step 4: Apply e^{-theta/2 * X1 Y0}
# Basis change: X1 -> Z1 with H, Y0 -> Z0 with Rx(pi/2)
circuit.h(1)
circuit.rx(np.pi/2, 0)
# Implement e^{i (-theta/2) Z1 Z0} = CNOT_{1,0} Rz(-2 * (-theta/2)) CNOT_{1,0}
circuit.cx(1, 0)
circuit.rz(theta, 0)  # -2 * (-theta/2) = theta
circuit.cx(1, 0)
# Inverse basis change
circuit.rx(-np.pi/2, 0)
circuit.h(1)

# Display the circuit
print(circuit)

        ┌───┐   ┌───┐┌───┐┌──────────┐┌───┐   ┌───┐    ┌─────────┐┌───┐»
q_0: ───┤ X ├───┤ H ├┤ X ├┤ Rz(-0.1) ├┤ X ├───┤ H ├────┤ Rx(π/2) ├┤ X ├»
     ┌──┴───┴──┐└───┘└─┬─┘└──────────┘└─┬─┘┌──┴───┴───┐└──┬───┬──┘└─┬─┘»
q_1: ┤ Rx(π/2) ├───────■────────────────■──┤ Rx(-π/2) ├───┤ H ├─────■──»
     └─────────┘                           └──────────┘   └───┘        »
«     ┌─────────┐┌───┐┌──────────┐
«q_0: ┤ Rz(0.1) ├┤ X ├┤ Rx(-π/2) ├
«     └─────────┘└─┬─┘└──┬───┬───┘
«q_1: ─────────────■─────┤ H ├────
«                        └───┘    


In [5]:
import prepare_MPS as mps
import numpy as np
from qiskit import BasicAer, execute

#Create Random MPS with size 4, bond dimension 4 and physical dimension 2 (qubits)
N=4
d=2
chi=4
phi_final=np.random.rand(chi)
phi_initial=np.random.rand(chi)
A=mps.create_random_tensors(N,chi,d)

#Create the circuit. The 'reg' register corresponds to the 'MPS' register in the picture above
qc, reg = mps.MPS_to_circuit(A, phi_initial, phi_final)

#Run the circuit on the statevector simulator
backend = BasicAer.get_backend("statevector_simulator")
job = execute(qc, backend)
result = job.result()
psi_out=result.get_statevector()

#Contract out the ancilla with the known state
psi_out=psi_out.reshape(d**N,chi)
exp=psi_out.dot(phi_final)

#Prepare the MPS classically
thr,_=mps.create_statevector(A,phi_initial,phi_final,qiskit_ordering=True)

#Compare the resulting vectors (fixing phase and normalization)
exp=mps.normalize(mps.extract_phase(exp))
thr=mps.normalize(mps.extract_phase(thr))

print("The MPS is \n {}".format(thr))
print("The statevector produced by the circuit is \n {}".format(exp))



ModuleNotFoundError: No module named 'prepare_MPS'

In [5]:
import prepare_MPS as mps
import numpy as np
from qiskit import BasicAer, execute

#Create Random MPS with size 4, bond dimension 4 and physical dimension 2 (qubits)
N=4
d=2
chi=4
phi_final=np.random.rand(chi)
phi_initial=np.random.rand(chi)
A=mps.create_random_tensors(N,chi,d)

#Create the circuit. The 'reg' register corresponds to the 'MPS' register in the picture above
qc, reg = mps.MPS_to_circuit(A, phi_initial, phi_final)

#Run the circuit on the statevector simulator
backend = BasicAer.get_backend("statevector_simulator")
job = execute(qc, backend)
result = job.result()
psi_out=result.get_statevector()

#Contract out the ancilla with the known state
psi_out=psi_out.reshape(d**N,chi)
exp=psi_out.dot(phi_final)

#Prepare the MPS classically
thr,_=mps.create_statevector(A,phi_initial,phi_final,qiskit_ordering=True)

#Compare the resulting vectors (fixing phase and normalization)
exp=mps.normalize(mps.extract_phase(exp))
thr=mps.normalize(mps.extract_phase(thr))

print("The MPS is \n {}".format(thr))
print("The statevector produced by the circuit is \n {}".format(exp))



ModuleNotFoundError: No module named 'prepare_MPS'

In [7]:
from qiskit import ClassicalRegister

N=5
chi=2

#The following is the standard representation of a GHZ state in terms of MPS
phi_initial=np.array([1,1])
phi_final=np.array([1,1])
T=np.zeros((d,chi,chi))
T[0,0,0]=1
T[1,1,1]=1
A=[]
for _ in range(N):
    A.append(T)
    

#Create the circuit, store the relevant wavefunction is register 'reg' and measure
qc, reg = mps.MPS_to_circuit(A, phi_initial, phi_final)
creg=ClassicalRegister(N)
qc.add_register(creg)
qc.measure(reg,creg)

#Run on a simulator
backend = BasicAer.get_backend("qasm_simulator")
job = execute(qc, backend)
result = job.result()
counts = result.get_counts(qc)
print("\nTotal counts are:",counts)

NameError: name 'd' is not defined

In [9]:
from qiskit import QuantumCircuit 

qc = QuantumCirtcuit(2)

qc.h(0)
qc.cx(0, 1)

qc.draw("mpl")

NameError: name 'QuantumCirtcuit' is not defined